# StructuredOutputParser

StructuredOutputParser는 LLM에 대한 답변을 `dict` 형식으로 구성하고, key/value 쌍으로 여러 필드를 반환하고자 할 때 유용하게 사용할 수 있습니다. 

## 장점
낮은 버전의 python에서도 사용 가능(Pydantic/JSON 파서는 문법적으로 Syntactic Sugar)

## 참고 사항
로컬 모델의 경우 `Pydantic` 파서가 동작하지 않는 상황이 빈번하게 발생할 수 있습니다. 이러한 경우, 대안으로 StructuredOutputParser를 사용하는 것이 좋은 해결책이 될 수 있습니다.


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Bedrock 설정 및 Util 함수 설정
import sys
from langchain_aws import ChatBedrockConverse
import os
from Util.stream_utils import print_stream_content, get_stream_content

# 객체 생성
llm = ChatBedrockConverse(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    model="apac.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="ap-northeast-2",
)

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [4]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate

- `ResponseSchema` 클래스를 사용하여 사용자의 질문에 대한 답변과 사용된 소스(웹사이트)에 대한 설명을 포함하는 응답 스키마를 정의합니다.
- `StructuredOutputParser`를 `response_schemas`를 사용하여 초기화하여, 정의된 응답 스키마에 따라 출력을 구조화합니다.


In [5]:
# 사용자의 질문에 대한 답변
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
]
# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

이제 응답이 어떻게 포맷되어야 하는지에 대한 지시사항이 포함된 문자열을 받게 되며(schemas), 정의된 스키마를 프롬프트에 삽입합니다.

In [6]:
# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)

In [7]:
model = llm  # Bedrock 모델 초기화
chain = prompt | model | output_parser  # 프롬프트, 모델, 출력 파서를 연결

In [ ]:
# 대한민국의 수도가 무엇인지 질문합니다.
chain.invoke({"question": "대한민국의 수도는 어디인가요?"})

dict

`chain.stream` 메소드를 사용하여 "세종대왕의 업적은 무엇인가요?" 라는 질문에 대한 스트림 응답을 받습니다.

In [9]:
for s in chain.stream({"question": "세종대왕의 업적은 무엇인가요?"}):
    # 스트리밍 출력
    print(s)

{'answer': '세종대왕의 주요 업적은 다음과 같습니다:\n\n1. **한글 창제** - 1443년 훈민정음(한글)을 창제하여 백성들이 쉽게 글을 읽고 쓸 수 있게 했습니다.\n\n2. **과학기술 발전** - 측우기, 해시계, 물시계 등 각종 과학기구를 발명하고 천문학을 발달시켰습니다.\n\n3. **영토 확장** - 4군 6진을 개척하여 북방 영토를 확장했습니다.\n\n4. **법전 정비** - 경국대전의 기초가 되는 각종 법령을 정비했습니다.\n\n5. **농업 발전** - 농사직설을 편찬하여 농업기술을 발전시켰습니다.\n\n6. **음악 발전** - 아악과 향악을 정리하고 음악 이론을 체계화했습니다.\n\n7. **의학 발전** - 향약집성방, 의방유취 등 의학서를 편찬했습니다.', 'source': '한국사 교육과정 및 국사편찬위원회 자료 기반 일반적 역사 지식'}
